In [33]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
import plotly.express as px

El dataset esta compuesto por una variable de tiempo, la inversion en diferentes medios, y las ventas en esa fecha. 

In [34]:
data = pd.read_csv('mmm.csv')
data

,Date,TV,Radio,Banners,Sales
0,2018-01-07,13528.10,0.00,0.00,9779.80
1,2018-01-14,0.00,5349.65,2218.93,13245.19
2,2018-01-21,0.00,4235.86,2046.96,12022.66
3,2018-01-28,0.00,3562.21,0.00,8846.95
4,2018-02-04,0.00,0.00,2187.29,9797.07
...,...,...,...,...,...
195,2021-10-03,0.00,0.00,1691.68,9030.17
196,2021-10-10,11543.58,4615.35,2518.88,15904.11
197,2021-10-17,0.00,4556.16,1919.19,12839.29
198,2021-10-24,0.00,0.00,1707.65,9063.45


No tenemos datos null en el dataset.

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     200 non-null    object 
 1   TV       200 non-null    float64
 2   Radio    200 non-null    float64
 3   Banners  200 non-null    float64
 4   Sales    200 non-null    float64
dtypes: float64(4), object(1)
memory usage: 7.9+ KB


Se puede ver que no tenemos inversion para: 

- tv en mas del 50% de las semanas
- Radio en mas del 50% de las semanas
- Banner tenemos inversion apartir del 25% de las semanas


In [36]:
data.describe()

,TV,Radio,Banners,Sales
count,200.000000,200.000000,200.000000,200.000000
mean,2946.207650,2213.585050,1520.722550,10668.141500
std,4749.646908,2505.967886,870.764354,2700.706683
min,0.000000,0.000000,0.000000,4532.330000
25%,0.000000,0.000000,1657.195000,8396.942500
50%,0.000000,0.000000,1918.990000,10853.105000
75%,7938.527500,4624.027500,2069.767500,12566.995000
max,13901.550000,7696.220000,2518.880000,17668.340000


la inversion en tv es mas alto que radio y a su vez este es mas alto que el de banner

In [37]:
col_medios = ['TV', 'Radio', 'Banners']
data_melt = pd.melt(data, id_vars=['Date'], value_vars=col_medios, var_name='Medio', value_name='Inversion')

In [38]:
fig = px.pie(data_melt, 
            values='Inversion', 
            names='Medio',
            title='Inversiones por medio')
fig.show()

## El punto es: obtener el valor de contribucion de cada medio, para generar ventas y saber cual seria el mejor monto de inversion para cada medio.

### Formula

$Ventas = f(TV) + g(Radio) + h(Banners) + Base$

la Base es la cantidad de ventas base que se tiene sin ninguna inversion, es decir el valor de la variable $y$ cuando $x(inversion)$ es $0$

Cosas a investigar dichas en el blog: 
- Contribucion Aditiva
- Modelos aditivos

Preparando el dataset

In [39]:
data = data.set_index('Date')
data.head()

,TV,Radio,Banners,Sales
Date,,,,
2018-01-07,13528.1,0.00,0.00,9779.80
2018-01-14,0.0,5349.65,2218.93,13245.19
2018-01-21,0.0,4235.86,2046.96,12022.66
2018-01-28,0.0,3562.21,0.00,8846.95
2018-02-04,0.0,0.00,2187.29,9797.07


In [40]:
X = data.drop(['Sales'], axis=1)
X.head()

,TV,Radio,Banners
Date,,,
2018-01-07,13528.1,0.00,0.00
2018-01-14,0.0,5349.65,2218.93
2018-01-21,0.0,4235.86,2046.96
2018-01-28,0.0,3562.21,0.00
2018-02-04,0.0,0.00,2187.29


In [41]:
y = data['Sales']
y.head()

Date
2018-01-07     9779.80
2018-01-14    13245.19
2018-01-21    12022.66
2018-01-28     8846.95
2018-02-04     9797.07
Name: Sales, dtype: float64

Implementando el modelo de regresion lineal

In [42]:
lr = LinearRegression()

In [43]:
lr.fit(X, y) # refit the model with the complete dataset
print('Coefficients:', lr.coef_)
print('Intercept:', lr.intercept_)

Coefficients: [0.35968382 0.48833246 1.2159193 ]
Intercept: 6678.396933606162


Desglose de las ventas por canales de contribucion

In [54]:
# filtrar por fecha en el index

data1010 = data.loc['2021-10-10']
tv = data1010['TV']
Radio = data1010['Radio']
Banners = data1010['Banners']

ventas_esperadas = lr.predict([[tv, Radio, Banners]])
ventas_originales = data1010['Sales']

print('Ventas esperadas:', ventas_esperadas)
print('Ventas originales:', ventas_originales)

Ventas esperadas: [16147.01594158]
Ventas originales: 15904.11


/home/dsandovalflavio/anaconda3/envs/ProyectMMM/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

